# 遷移學習- transfer learning

In [ ]:
!gdown '1_pPLiZTiSPuXwnivKlhDIqs2pMSNsAPy' --output selfFlower.zip
!unzip selfFlower.zip

## 預訓練模型

預訓練模型

• 預訓練模型是他人已經建立完成的模型，只要取得預訓練模型，就能立刻開始使用該模型進行預測。

• ImageNet：

自2010年開始，ImageNet每年舉辦一次軟體競賽，即ILSVRC視覺辨識挑戰賽。在挑戰賽使用中的圖片超過1000000張，共分1000個類別，用於打分數的圖片有100000張。

Keras將研發團隊精心調校的模型及執行結果收集進來，一般使用者就不用自己訓練模型，可以直接套用，成為Keras內建的預訓練模型。

建立預訓練模型

• 載入預訓練模型的模組：
```
from keras.applications.應用模型 import 模型名稱
```
• 建立預訓練模型：

```
模型變數 = 模型名稱(weights=權重, include_top=布林值)
```

使用預訓練模型預測圖片

• 載入模組：

```
from keras.applications.應用模型 import preprocess_input,
decode_predictions
```

• 圖片預處理：

```
圖片變數 = preprocess_input(圖片變數)
```

• 進行預測：
```
預測變數 =模型變數.predict(圖片變數)
```
• 預測結果：
```
decode_predictions(預測變數, top=數值)[0]
```


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions

model = InceptionV3(weights='imagenet', include_top=True)

import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt

# 讀入 daisy1.jpg
img_path = 'daisy1.jpg'
img = image.load_img(img_path, target_size=(299, 299))
plt.imshow(img)
plt.show()

img_array = image.img_to_array(img)
print(img_array.shape)

img_array = np.expand_dims(img_array, axis=0)
inputs = preprocess_input(img_array) # 圖片預處理只能做1次
print(inputs.shape)

pred = model.predict(inputs)
results = decode_predictions(pred, top=5)
print(f'模型預測結果:{results[0][0][1]}, 信心度:{results[0][0][2]*100:.2f}%')

results

pred[0][:10]

len(pred[0])

Gradio

In [ ]:
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import gradio as gr
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions
import cv2

model = ResNet50(weights='imagenet', include_top=True)

def guess(img):
  img = cv2.resize(img, (224, 224))
  img = np.expand_dims(img, axis=0) #展成 4 維 (1, 224, 224, 3)
  inputs = preprocess_input(img) # 圖片預處理只能做1次
  pred = model.predict(inputs)
  results = decode_predictions(pred, top=5)
  text =''
  for i in range(5):
    text += f'{results[0][i][1]}, confidence:{results[0][i][2]*100:.2f}%\n'
  return text

demo = gr.Interface(fn=guess, inputs="image", outputs="text")
demo.launch(share=True)

In [ ]:
demo.close()

## 遷移學習

• 深度學習最常見的障礙有兩個：：

• **有海量的參數需要訓練**：VGG16及VGG19達到一億四千萬個。

• **需要海量資料進行訓練**：ILSVRC挑戰賽使用的圖片共分1000個類別，圖片超過1,000,000張。

• **遷移學習是將一個場景中學到的知識「遷移」到另一個場景中。以貓狗分類的模型為例，將其遷移到其他相似的任務上，例如用來分辨麻雀與燕子，因為它們都是以圖片來辨識物件，所以屬於相同領域，抽取特徵的方法是相同的**。

遷移學習蒐集資料：花朵資料集

• 花朵資料集為Tensorflow官網提供，包含雛菊(daisy)、蒲公英
(dandelion)、玫瑰(rose)、向日葵(sunflower)、鬱金香(tulip) 5種花朵圖片，每種花朵圖片數量在600張到900張之間。為彰顯遷移學習效果，我們每種圖片僅使用50張。

In [ ]:
# 蒐集資料：花朵資料集
!wget http://download.tensorflow.org/example_images/flower_photos.tgz
!tar -xvf "flower_photos.tgz"

解壓縮會有LICENSE.txt 要把它刪除.

In [ ]:
!rm /content/flower_photos/LICENSE.txt

### 讀入圖片

In [ ]:
# 讀入圖片,每類讀50張.

dict_labels = {"daisy":0, "dandelion":1, "roses":2, "sunflowers":3, "tulips":4}


### 遷移學習資料預處理

• 串列轉Numpy陣列：
```
images = np.array(images)
labels = np.array(labels)

```
• 特徵值標準化：
```
trainX = preprocess_input(images)
```
• 標籤資料One-Hot編碼：
```
trainY = to_categorical(labels)
```

In [ ]:
# 資料預處理模組
import os, cv2, glob

images = []
labels = []

for folder in glob.glob(r'/content/flower_photos/*'):
  print(f'讀取資料夾:{folder}')
  i = 1
  for f in os.listdir(folder):
    if i > 50: break
    img = cv2.imread(os.path.join(folder, f))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (299, 299))
    if img is not None:
      images.append(img)
      labels.append(dict_labels[folder.split('/')[-1]])
      i += 1

print('圖片讀取完成')

len(images)

print(labels)

plt.imshow(image[0])

In [ ]:
# 資料預處理
import numpy as np
from keras.applications.inception_v3 import preprocess_input
from keras.utils import to_categorical

images = np.array(images)
labels = np.array(labels)

trainX = preprocess_input(images)
trainY = to_categorical(labels)

### 花朵辨識卷積神經網路模型

In [ ]:
# 花朵辨識卷積神經網路模型
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model2 = Sequential([
    Conv2D(8, (5, 5), padding='same',activation='relu', input_shape=(299, 299, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (5, 5), padding='same',activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Conv2D(32, (5, 5), padding='same',activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Flatten(),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

model2.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model2, show_shapes=True)

In [ ]:
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

model2.compile(loss=CategoricalCrossentropy(),
               optimizer=Adam(),
               metrics=['accuracy'])

model2.fit(trainX, trainY, batch_size=10, epochs=10, verbose=2)

run --> run selection 執行選擇區域.

In [ ]:
model2.save('flower_model.keras')

loss, acc = model2.evaluate(trainX, trainY)
print(f'loss:{loss:.4f}, accuracy:{acc*100:.2f}')

In [ ]:
#秀出預測結果
def show_imgs_labs_preds(images, labels, predictions,start_id, num=10):
    plt.figure(figsize=(12, 14))
    if num>25: num=25
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[start_id])
        if( len(predictions) > 0 ) :
            title = 'ai = ' + str(predictions[start_id])
            title += (' (o)' if predictions[start_id]==labels[start_id] else ' (x)')
            title += '\nlabel = ' + str(labels[start_id])
        else :
            title = 'label = ' + str(labels[start_id])
        ax.set_title(title,fontsize=12)
        ax.set_xticks([]);ax.set_yticks([])
        start_id+=1
    plt.show()

In [ ]:
# 用自己圖片進行預測
import glob, cv2

files = glob.glob('/content/selfFlower/*.jpg')
testX = []
testY = []
for f in files:
  img = cv2.imread(f)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (299, 299))
  testX.append(img)
  testY.append(dict_labels[f.split('/')[-1][:-5]])

testX = np.array(testX)
testY = np.array(testY)

In [ ]:
testX.shape

testY

testX2 = preprocess_input(testX)
preds = np.argmax(model2.predict(testX2), axis=1)
show_imgs_labs_preds(testX, testY, preds, start_id=0, num=10)

### 遷移學習建立模型


• 載入預訓練模型 (不含輸入層)：
```
預訓練變數 = 模型名稱(weights='imagenet', include_top=False,
pooling='max|avg')
```

• 預訓練模型不訓練：
```
預訓練變數.trainable = False
```
• 加入預訓練模型：
```
模型變數.add(預訓練變數)
```
• 加入輸出層：
```
模型變數.add(Dense(數值, activation='softmax'))
```

### 花朵辨識遷移學習模型

In [ ]:
# 花朵辨識遷移學習模型
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense

• 載入預訓練模型 (不含輸入層)：
```
預訓練變數 = 模型名稱(weights='imagenet', include_top=False, pooling='max|avg')
```

In [ ]:
inception = InceptionV3(weights='imagenet', include_top=False, pooling='avg')

#預訓練模型不訓練, 凍結參數
inception.trainable = False
# 組裝模型
model3 = Sequential()
model3.add(inception)
model3.add(Dense(5, activation='softmax'))

model3.summary()

In [ ]:
# 訓練模型
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(trainX, trainY, batch_size=10, epochs=10)

In [ ]:
# 儲存模型
model.save('transfer_model.h5')

In [ ]:
# 用自己圖片進行預測
preds2 = np.argmax(model3.predict(testX2), axis=1)
show_imgs_labs_preds(testX, testY, preds2, start_id=0, num=10)

## Gradio

In [ ]:
#　@title 多模型練習 gradio

import gradio as gr
import numpy as np
import cv2
import matplotlib.pyplot as plt

def guess(img, model='model2'):
  img = cv2.resize(img, (299, 299))
  img = np.expand_dims(img, axis=0)
  inputs = preprocess_input(img)
  if model == 'model2':
    pred = model2.predict(inputs)
  else:
    pred = model3.predict(inputs)
  res = np.argmax(pred, axis=1)
  return class_names[res[0]]

demo = gr.Interface(guess,
                    inputs=[gr.Image(height=299, width=299),
                            gr.Dropdown(['model2', 'model3'], value='model3')],
                    outputs='text')
demo.launch(share=True)

In [ ]:
demo.close()